In [2]:
from enum import StrEnum

from pydantic import BaseModel, Field
from pydantic_ai import Agent,RunContext

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
Without Graph:
1. Tool A does something that writes do DB

In [5]:



def greeting() -> str:
    """greeting Hello, This tool is to say 'Hello'."""
    print("*********** Called function say_hello .")
    return "hey brooo"


# Create an agent
agent = Agent(
    model="google-gla:gemini-2.0-flash-exp",
    system_prompt=(
        "You are a very friendly assistant. Always use the `greeting` function to start the response "
        "Do not add any extra text or explanations."
    ),
    tools=[greeting],
)

In [6]:
res = await agent.run("hellooo")

In [7]:
res.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are a very friendly assistant. Always use the `greeting` function to start the response Do not add any extra text or explanations.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='hellooo', timestamp=datetime.datetime(2025, 1, 27, 8, 36, 17, 117837, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='greeting', args=ArgsDict(args_dict={}), tool_call_id=None, part_kind='tool-call')], model_name='gemini-2.0-flash-exp', timestamp=datetime.datetime(2025, 1, 27, 8, 36, 18, 783322, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='greeting', content='hey brooo', tool_call_id=None, timestamp=datetime.datetime(2025, 1, 27, 8, 36, 18, 786157, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'),
 ModelResponse(parts=[TextPart(content='hey brooo\n', part_kind='text')], model_name='gemini-2

In [8]:
res.data

'hey brooo\n'